In [ ]:
import torch
import numpy as np
import torch.nn as nn

In [29]:
import importnb
with __import__('importnb').Notebook(): 
    from scaled_dot_product_attention import ScaledDotProductAttention

## SelfAttentionの実行

### 適当なデータの作成
---
新しくデータkを作る。kは***(B,T,d_model)***のshapeを持つテンソルである。


### SelfAttentionにする
---
q,k,vを同じテンソルにすることでSelfAttentionにする。


### forwardで計算を行う。
---
呼び出したspa.forward()によって計算を行う。
この時spaの初期化で与えるd_kには本来d_modelをhead数で割った値が入る(デフォルトだと512/8で64)

In [30]:
###SelfAttentionの実行
k = torch.Tensor([
[
    [1,-2],
    [-8,7],
    [10,9],
    [13,14]
],
[
    [3,4],
    [5,6],
    [8,9],
    [10,11]
],
[
    [1,1],
    [2,4],
    [5,7],
    [3,6]
]
])

In [31]:
###SelfAttentionにするためにqとvとkを一緒にする
q = k
v = k
spa = ScaledDotProductAttention(64)
attention_weight = spa.forward(q,k,v)

In [32]:
attention_weight

tensor([[[ 2.8651,  0.8840],
         [-8.0000,  7.0000],
         [12.9997, 13.9996],
         [13.0000, 14.0000]],

        [[ 9.6417, 10.6417],
         [ 9.8747, 10.8747],
         [ 9.9718, 10.9718],
         [ 9.9895, 10.9896]],

        [[ 3.3903,  5.4370],
         [ 4.2456,  6.4677],
         [ 4.7578,  6.8626],
         [ 4.5564,  6.7296]]])